# Understand param of keras mode.summary()

* keras `model.summary()` prints `Param`, but how to understand the `Param` ?

# 1. basic nn

In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

model = Sequential() # 顺序模型

# 输入层
model.add(Dense(7, input_shape=(4,)))  # Dense就是常用的全连接层
model.add(Activation('sigmoid')) # 激活函数

# 隐层
model.add(Dense(13))  # Dense就是常用的全连接层
model.add(Activation('sigmoid')) # 激活函数

# 输出层
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 7)                 35        
_________________________________________________________________
activation_4 (Activation)    (None, 7)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 13)                104       
_________________________________________________________________
activation_5 (Activation)    (None, 13)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 70        
_________________________________________________________________
activation_6 (Activation)    (None, 5)                 0         
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


In [3]:
for layer in model.layers:
    print(layer.name, layer.count_params(), layer.output_shape)

dense_4 35 (None, 7)
activation_4 0 (None, 7)
dense_5 104 (None, 13)
activation_5 0 (None, 13)
dense_6 70 (None, 5)
activation_6 0 (None, 5)


**Analysis**
* 35 = 7*(4+1)
* 104 = 13*(7+1)
* 70 = 5*(13+1)

# 2. cnn

In [14]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Convolution2D as Conv2D
from keras.layers import MaxPooling2D
from keras import backend as K
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


digit = load_digits()

data_x = digit.data
data_y = digit.target


x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=42)

batch_size = 128
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 8, 8


if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (1437, 8, 8, 1)
1437 train samples
360 test samples


In [15]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 2),
                 input_shape=input_shape))
convout1 = Activation('relu')
model.add(convout1)

model.add(Conv2D(64, (2, 3), activation='relu'))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 6, 7, 32)          224       
_________________________________________________________________
activation_11 (Activation)   (None, 6, 7, 32)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 5, 5, 64)          12352     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 4, 4, 64)          16448     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 256)               0         
__________

In [17]:
for layer in model.layers:
    print(layer.name, layer.count_params(), layer.output_shape)

conv2d_13 224 (None, 6, 7, 32)
activation_11 0 (None, 6, 7, 32)
conv2d_14 12352 (None, 5, 5, 64)
conv2d_15 16448 (None, 4, 4, 64)
max_pooling2d_5 0 (None, 2, 2, 64)
dropout_9 0 (None, 2, 2, 64)
flatten_5 0 (None, 256)
dense_15 32896 (None, 128)
dropout_10 0 (None, 128)
dense_16 1290 (None, 10)


**total_params = (filter_height * filter_width * input_image_channels + 1) * number_of_filters**


first layer
* 224 = (2*3*1+1)*32 # 32个2x3的卷积核（+1 bias）

second layer
* Conv2D(64, (2, 3), activation='relu')
* (2*3*32+1)*64 = 12352

third layer
* Conv2D(64, (2, 2), activation='relu')
* (2*2*64+1)*64


# 3. dcnn

In [37]:
from keras.layers import Input, Deconvolution2D, Conv2DTranspose
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from sklearn.datasets import load_digits
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Convolution2D as Conv2D
from keras.layers import MaxPooling2D
from keras import backend as K


# create DCNN
model = Sequential()

model.add(Deconvolution2D(5, kernel_size=(2,2), activation='relu', input_shape=(40, 110, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Deconvolution2D(3, kernel_size=(2,2), activation='relu'))
model.add(Flatten())

model.add(Dense(158, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [38]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_9 (Conv2DTr (None, 41, 111, 5)        25        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 20, 55, 5)         0         
_________________________________________________________________
conv2d_transpose_10 (Conv2DT (None, 21, 56, 3)         63        
_________________________________________________________________
flatten_9 (Flatten)          (None, 3528)              0         
_________________________________________________________________
dense_23 (Dense)             (None, 158)               557582    
_________________________________________________________________
dropout_14 (Dropout)         (None, 158)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 10)                1590      
Total para

First Deconvolution2D
* (2*2*1+1)*5 = 25

Second Deconvolution2D
* (2*2*5+1)*3